In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
import string
from nltk import tokenize
from selenium import webdriver
import os
from textblob import TextBlob
from google_trans_new import google_translator
translator = google_translator()
from collections import defaultdict
from pandas.core.common import flatten
from collections import Counter
from nltk.corpus import stopwords
import numpy as np
import short_url

In [2]:
df = pd.read_csv('dataframe_data steward.csv')

In [3]:
df

,id,title,company,link,description
0,DSP_0,Data steward / analyst,JLL,https://nl.indeed.com/rc/clk?jk=32d6b156cbb9d2...,"<div class=""jobsearch-JobComponent-description..."
1,DSP_1,Global Process Director,Cargill,https://nl.indeed.com/rc/clk?jk=f468bb81155e0a...,"<div class=""jobsearch-JobComponent-description..."
2,DSP_2,Human Resources Generalist,THE RENEWAL WORKSHOP,https://nl.indeed.com/rc/clk?jk=cb367910062ed1...,"<div class=""jobsearch-JobComponent-description..."
3,DSP_3,Data Engineer,Optiver,https://nl.indeed.com/rc/clk?jk=1bdc22b275c924...,"<div class=""jobsearch-JobComponent-description..."
4,DSP_4,Senior Data Management Analyst Mortgages,ING,https://nl.indeed.com/rc/clk?jk=6ac940b5c663f1...,"<div class=""jobsearch-JobComponent-description..."
5,DSP_5,Product Owner Data Management Mortgages,ING,https://nl.indeed.com/rc/clk?jk=deca9994e792aa...,"<div class=""jobsearch-JobComponent-description..."
6,DSP_6,RA Manager,RB,https://nl.indeed.com/rc/clk?jk=76a34a12579b88...,"<div class=""jobsearch-JobComponent-description..."
7,DSP_7,Data Engineer,Carrierecafe,https://nl.indeed.com/rc/clk?jk=f70cb6470c9974...,"<div class=""jobsearch-JobComponent-description..."
8,DSP_8,Inbound Marketer,24sessions,https://nl.indeed.com/rc/clk?jk=8376556adb3173...,"<div class=""jobsearch-JobComponent-description..."
9,DSP_9,Data Analist,Carrierecafe,https://nl.indeed.com/rc/clk?jk=766d1bdeda95fb...,"<div class=""jobsearch-JobComponent-description..."


In [4]:
def remove_btags(bulletpoint):
    # remove any bold text from the bulletpoints
    try: 
        bulletpoint.b.unwrap()
    except:
        pass
    return bulletpoint 

In [5]:
def p_bullets(description, bulletpoint, str_bullet):
    # find <p> version of text in description
    string = description.find('p', text=bulletpoint.text)
    if string:
    # loop over all preceding p tags
        for p_string in string.find_all_previous('p'):
            p_string = f'<li>{p_string}</li>'
            # find the one that is not already in the bulletspoint
            if p_string not in str_bullet:
                head = re.sub('<[^<]+?>', '', p_string)
                break
    else:
        head = None
    return head

In [6]:
def normal_bullets(bulletpoint):
    # head of bulletpoints is previous <p> tag
    if bulletpoint.find_previous('p') == None:
        head = None
    else:
        head = bulletpoint.find_previous('p').text.replace('\n', '')
        # if head is too long, it is normal text, not the head
        if len(head) > 60 or len(head) == 0:
            # check if head is h3, h2 or b
            try:
                head = bulletpoint.find_previous('h3').text.replace('\n', '')
            except:
                try:
                    head = bulletpoint.find_previous('h2').text.replace('\n', '')
                except:
                    try:
                        head = bulletpoint.find_previous('b').text.replace('\n', '')
                    except:
                        head = None
    return head

In [7]:
def br_bullets(bulletpoint):
    """
    When bulletpoints are inside <br> tags
    """
    previous = [sibling.nextSibling for sibling in bulletpoint.find_previous('br').find_previous_siblings()]
    if len(previous) == 0:
        head = None
    else:
        if isinstance(previous[0], str):
            head = previous[0].replace('\n', '')
        else:
            head = None
    return head

In [8]:
def translate(text):
    return translator.translate(text, lang_tgt='en').lower()

In [9]:
def bulletpoints_dict(dataframe):
    all_bulletpoints = defaultdict(list)
    for index, description in enumerate(dataframe['description']):
        bulletpoints = BS(description, 'html.parser').findAll('li')
        bs4_description = BS(description, 'html.parser')

        str_bullets = list(map(str, bulletpoints))
        # if there are no bulletpoints:
        if len(bulletpoints) == 0:
            pass
        else:
            # detect language
            lang = TextBlob(bulletpoints[0].text[:100]).detect_language()
             # for each bulletpoint
            for item in bulletpoints:
                # remove bold text
                item = remove_btags(item)
                # check for <p> case
                if '<p>' in str(item):
                    head = p_bullets(bs4_description, item, str_bullets)
                # if only br text
                elif item.find_previous('p') == None:
                    head = br_bullets(item)
                else: 
                    head = normal_bullets(item)
                # add head as key and bulletpoint as value
                if head == None:
                     head = 'no head'
                else:
                    # if head is too long, skip it
                    if len(head) > 60:
                        pass
                    else:
                        # if language is dutch, translate to english
                        if lang == 'nl':
                            head = translate(head)
                            point = translate(item.text.replace('\n', ''))
                        else:
                            head = head.lower()
                            point = item.text.replace('\n', '').lower()

                        headlist = (dataframe.iloc[index]['id'], head)
                        all_bulletpoints[headlist].append(point)
                        
    return all_bulletpoints

In [10]:
all_bullet_dict = bulletpoints_dict(df)

In [11]:
all_bullet_dict

defaultdict(list,
            {('DSP_4',
              'an overview of your responsibilities'): ['together with the mortgages tribe you ensure that the mortgages requirements are met', 'you act as a subject matter expert towards the tribe mortgages regarding data delivery (data profiling, data mapping, analysis and data governance)', 'you align with the business stakeholders the scope of the data that needs to be delivered (system of record to the data users)', 'jointly with the delivery squads and mortgages stakeholders, you define the data in the datamart. these business terms are aligned with the descriptions provided by and aligned with the “ing esperanto”, which is the model ing is using for data definitions and physical translations', 'you ensure that local ing nl mortgages data are translated to the banking datawarehouse model (ing esperanto)', 'you draw up conceptional, logical, and dimensional data models and you are a sparring partner for engineers for the translation of thes

In [12]:
green = ['you have', 
         'you’ll have',
         'you need',
         'you’ll need',
         'you are',
         'you offer',
         'you bring',
         'about you',
         'we ask', 
         'we expect', 
         'require', 
         'skills', 
         'knowledge', 
         'experience', 
         'education',
         'candidate', 
         'profile',
         'looking for',
         'qualification',
         'recognize yourself']

In [13]:
def requirement(item, green):
    res = False
    if item:
        if any(word in item for word in green):
            res = True
    return res

In [14]:
def select_requirements(dictionary, green_list):
    requirements_dict = {k: v for k, v in dictionary.items() if requirement(k[1], green)}
    return requirements_dict     

In [15]:
req = select_requirements(all_bullet_dict, green)
req

{('DSP_4',
  'your profile and skills'): ['senior knowledge and experience with the banking datawarehouse model (bdwm)', 'senior knowledge and experience with loan products, especially mortgages in the dutch context', 'senior knowledge and experience with laws and regulations around financial institutions (local and international)', 'senior experience with data analysis (sql)', 'knowledge of and experience in data management, data modelling (dimensional) and data warehouse technology', 'experience in data governance (data definitions, data quality and data lineage)', 'strong conceptual and analytical skills', 'excellent communication skills', 'excellent command in dutch and english (verbal and in writing)', 'team player: focus on delivery through collaboration, ability to unite people to work towards the same purpose', 'proven experience or strong affinity with the ing agile way of working', 'enthusiastic personality, who motivates the squad performance and supports the agile way of wo

In [16]:
#%store req